<a href="https://colab.research.google.com/github/chaiminwoo0223/Deep-Learning/blob/main/09%20-%20VGGNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGGNet

- 2014 ILSVRC 2nd place
- VGG-16
- Convolution layer
- Maxpooling layer
- Fully connected layer

![대체 텍스트](https://qph.fs.quoracdn.net/main-qimg-e657c195fc2696c7d5fc0b1e3682fde6)

# Import

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os

# 데이터 다운로드

In [2]:
!rm -r images

# 이미지 파일을 저장할 폴더를 생성
try:
    os.mkdir("images")
    os.mkdir("images/dogs")
    os.mkdir("images/cats")
except:
    pass

# 이미지들을 저장한 위치에 다운로드
# images/dogs 밑에 2개
!wget https://i.kinja-img.com/gawker-media/image/upload/s--WFkXeene--/c_scale,f_auto,fl_progressive,q_80,w_800/ol9ceoqxidudap8owlwn.jpg -P images/dogs
!wget https://www.rspcansw.org.au/wp-content/uploads/2017/08/50_a-feature_dogs-and-puppies_mobile.jpg -P images/dogs
# images/cats 밑에 2개
!wget https://www.catster.com/wp-content/uploads/2018/05/A-gray-cat-crying-looking-upset.jpg -P images/cats
!wget https://www.scarymommy.com/wp-content/uploads/2018/01/c1.jpg?w=700 -P images/cats

rm: cannot remove 'images': No such file or directory
--2023-04-10 11:55:34--  https://i.kinja-img.com/gawker-media/image/upload/s--WFkXeene--/c_scale,f_auto,fl_progressive,q_80,w_800/ol9ceoqxidudap8owlwn.jpg
Resolving i.kinja-img.com (i.kinja-img.com)... 151.101.194.166, 151.101.66.166, 151.101.2.166, ...
Connecting to i.kinja-img.com (i.kinja-img.com)|151.101.194.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36509 (36K) [image/jpeg]
Saving to: ‘images/dogs/ol9ceoqxidudap8owlwn.jpg’

ol9ceoqxidudap8owlw 100%[===================>]  35.65K  --.-KB/s    in 0.001s  

2023-04-10 11:55:34 (48.9 MB/s) - ‘images/dogs/ol9ceoqxidudap8owlwn.jpg’ saved [36509/36509]

--2023-04-10 11:55:34--  https://www.rspcansw.org.au/wp-content/uploads/2017/08/50_a-feature_dogs-and-puppies_mobile.jpg
Resolving www.rspcansw.org.au (www.rspcansw.org.au)... 101.0.86.38
Connecting to www.rspcansw.org.au (www.rspcansw.org.au)|101.0.86.38|:443... connected.
HTTP request sent, awaiting 

# Hyperparameter

In [3]:
batch_size = 1
learning_rate = 0.0002
num_epoch = 100

# DataLoader

In [4]:
img_dir = "./images"
img_data = dset.ImageFolder(img_dir, transforms.Compose([
                            transforms.Resize(256),            # 이미지 크기 = 256*256
                            transforms.RandomResizedCrop(224), # 256*256 이미지의 랜덤한 위치에서 224*224 크기만큼 샘플링
                            transforms.RandomHorizontalFlip(), # 랜덤한 확률로 이미지를 죄우반전
                            transforms.ToTensor()
]))

train_loader = data.DataLoader(img_data, batch_size=batch_size, shuffle=True, num_workers=2)

# Model

## 1.Basic Blocks
- 모델에 반복되는 부분이 많기 때문에, 이를 함수로 만들어 단순화

In [5]:
# 컨볼루션 연산이 2번 연속하는 경우
# 컴볼루션 - 활성화함수 - 컨볼루션 - 활성화함수 - 풀링
def conv_2_block(in_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

# 컨볼루션 연산이 3번 연속하는 경우
# 컴볼루션 - 활성화함수 - 컨볼루션 - 활성화함수 - 컨볼루션 - 활성화함수 - 풀링
def conv_3_block(in_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

## 2.VGG Model

In [6]:
# VGG 네트워크
class VGG(nn.Module):
    def __init__(self, base_dim, num_classes=2):
        super(VGG, self).__init__()
        self.feature = nn.Sequential(
            conv_2_block(3,base_dim),
            conv_2_block(base_dim,2*base_dim),
            conv_3_block(2*base_dim,4*base_dim),
            conv_3_block(4*base_dim,8*base_dim),
            conv_3_block(8*base_dim,8*base_dim)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(8*base_dim*7*7,100),
            nn.ReLU(True),
            nn.Linear(100,20),
            nn.ReLU(True),
            nn.Linear(20,num_classes)
        )
    def forward(self,x):
        x = self.feature(x)
        x = x.view(x.size(0),-1)
        x = self.fc_layer(x)
        return x

# Optimizer & Loss

In [7]:
# GPU가 사용 가능한 경우에는 device를 0번 GPU로 설정하고, 불가능한 경우에는 CPU로 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
# VGG 클래스를 인스턴스화하고 지정한 장치에 올림
model = VGG(base_dim=16).to(device)

In [9]:
# 손실함수
loss_func = nn.CrossEntropyLoss()
# 최적화함수
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# 모델 자녀 노드의 이름과 모듈을 출력
for i in model.named_children():
    print(i)

('feature', Sequential(
  (0): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Sequential(


# Train

In [11]:
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)

        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()

    if i % 10 ==0:
        print(loss)

tensor(0.7425, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6560, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.0913, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.0755, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3825, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.9256, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0132, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6594, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.7406, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7112, device='cuda:0', grad_fn=<NllLossBackward0>)
